## Upgrade stored XGBoost models to new format

This notebook is split in two parts: The first, we run in the environment with xgboost version 0.90 to be able to read the old pickle files. We then store them with the `save_model` function into a format that can be read with later versions. We also evaluate the model on a small subset of our data and store the results.

The second part is run with the new version of xgboost and loads the stored models to check that they can be read. We also check that the results are the same as before.
Finally, we store the model again with the new version into the JSON format. 

### Make some test data

In [2]:
import sys
sys.path.append("../../")
import data_loading as dl

RUN = ["3"]
all_truth_filters = [
    "nue",
    "drt",
    # remove the ones below to save some time
    # "nc_pi0",
    # "cc_pi0",
    # "cc_nopi",
    # "cc_cpi",
    # "nc_nopi",
    # "nc_cpi",
]
rundata, mc_weights, data_pot = dl.load_runs(
    RUN,
    data="bnb",
    truth_filtered_sets=all_truth_filters,
    loadpi0variables=True,
    loadshowervariables=True,
    loadrecoveryvars=True,
    loadsystematics=True,
    load_lee=True,
    use_bdt=True
)

../../data_loading.py:1078: RuntimeWarning: overflow encountered in multiply
  mc_Q2 = -1 * (mc_q_E * mc_q_E - mc_q_px * mc_q_px - mc_q_py * mc_q_py - mc_q_pz * mc_q_pz)
../../data_loading.py:1078: RuntimeWarning: invalid value encountered in subtract
  mc_Q2 = -1 * (mc_q_E * mc_q_E - mc_q_px * mc_q_px - mc_q_py * mc_q_py - mc_q_pz * mc_q_pz)
../../data_loading.py:594: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * e1 * e2 * (1 - px1 * px2 - py1 * py2 - pz1 * pz2))
../../data_loading.py:594: RuntimeWarning: overflow encountered in float_scalars
  return np.sqrt(2 * e1 * e2 * (1 - px1 * px2 - py1 * py2 - pz1 * pz2))
/Users/trettin/miniconda/envs/python3LEE/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
../../data_loading.py:757: RuntimeWarning: invalid value encountered in true_divide
  df["proton_pz"] = np.where((mc_E_prot > 0), mc_pz_prot / mc_p_prot, 

Loading lee sample


In [6]:
mc_sample = rundata["mc"].sample(1000)

In [13]:
# Variables used in the pi0 BDTs
TRAINVAR = [
    "shr_score",
    "tksh_distance",
    "tksh_angle",
    "shr_tkfit_dedx_max",
    "trkfit",
    "trkpid",
    "subcluster",
    "shrmoliereavg",
    "trkshrhitdist2",
    "hits_ratio",
    "secondshower_Y_nhit",
    "secondshower_Y_vtxdist",
    "secondshower_Y_dot",
    "anglediff_Y",
    "CosmicIPAll3D",
    "CosmicDirAll3D",
]

# Variables used in the background BDTs
TRAINVARZP = [
    "shrmoliereavg",
    "shr_score",
    "trkfit",
    "subcluster",
    "CosmicIPAll3D",
    "CosmicDirAll3D",
    "secondshower_Y_nhit",
    "secondshower_Y_vtxdist",
    "secondshower_Y_dot",
    "anglediff_Y",
    "secondshower_V_nhit",
    "secondshower_V_vtxdist",
    "secondshower_V_dot",
    "anglediff_V",
    "secondshower_U_nhit",
    "secondshower_U_vtxdist",
    "secondshower_U_dot",
    "anglediff_U",
    "shr_tkfit_2cm_dedx_U",
    "shr_tkfit_2cm_dedx_V",
    "shr_tkfit_2cm_dedx_Y",
    "shr_tkfit_gap10_dedx_U",
    "shr_tkfit_gap10_dedx_V",
    "shr_tkfit_gap10_dedx_Y",
    "shrMCSMom",
    "DeltaRMS2h",
    "shrPCA1CMed_5cm",
    "CylFrac2h_1cm",
]

BDTVARS = TRAINVAR + TRAINVARZP

test_df = mc_sample[BDTVARS]
test_df


,shr_score,tksh_distance,tksh_angle,shr_tkfit_dedx_max,trkfit,trkpid,subcluster,shrmoliereavg,trkshrhitdist2,hits_ratio,...,shr_tkfit_2cm_dedx_U,shr_tkfit_2cm_dedx_V,shr_tkfit_2cm_dedx_Y,shr_tkfit_gap10_dedx_U,shr_tkfit_gap10_dedx_V,shr_tkfit_gap10_dedx_Y,shrMCSMom,DeltaRMS2h,shrPCA1CMed_5cm,CylFrac2h_1cm
entry,,,,,,,,,,,,,,,,,,,,,
2603,0.060575,18.913317,-4.230924e-01,3.801373,0.333333,-0.301138,11,12.420587,12.909738,0.686099,...,4.518741,-3.402823e+38,5.315952e+00,2.160177,-3.402823e+38,4.244851e+00,66.594582,2.284401,0.916764,0.265823
20534,0.164992,979.931641,-3.402823e+38,4.410536,0.278689,NaN,9,4.436848,-1.000000,1.000000,...,4.410536,3.194422e+00,5.783832e+00,3.542192,3.786885e+00,8.806034e+00,84.530724,1.634778,0.945253,0.380952
4992,0.059947,21.079191,-4.609286e-01,4.436085,0.241546,0.551929,3,6.491850,17.001177,0.769517,...,3.892681,3.675340e+00,1.527958e+00,3.244662,5.104007e+00,2.351186e+00,64.836075,0.754496,0.928969,0.842105
5985,0.464402,23.760128,-6.125049e-01,4.500453,0.554913,-0.687969,8,5.091989,5.137547,0.582868,...,3.329741,3.543460e+00,-3.402823e+38,4.762031,3.927620e+00,-3.402823e+38,83.340912,0.651804,0.996718,0.880000
29249,0.113901,55.102726,6.791703e-01,2.444770,0.640719,0.928477,3,6.599442,48.592247,0.164370,...,2.302901,2.772507e+00,2.326897e+00,2.783712,3.844409e+00,2.793659e+00,32.812866,2.275570,0.996513,0.103448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13765,0.008176,6.881827,3.203493e-01,4.140000,0.219057,0.948853,28,6.634148,6.404014,0.569033,...,4.586287,5.191947e+00,3.978160e+00,3.534937,4.625525e+00,4.039353e+00,120.146133,6.285063,0.808459,0.000000
7455,0.002178,136.825684,-3.402823e+38,2.176004,0.105548,NaN,22,7.937480,-1.000000,1.000000,...,0.152080,2.002775e+00,2.186585e+00,-0.008714,2.629586e+00,2.188087e+00,208.518311,2.094876,0.727142,0.101770
10608,0.030994,7.351628,3.889772e-01,2.261708,0.207237,0.648987,9,10.511190,2.278313,0.568224,...,4.155707,1.508572e+00,2.357685e+00,2.033264,4.154612e+00,3.306931e+00,33.613930,2.566018,0.688182,0.147287


In [16]:
import pandas as pd


test_df.to_csv('test_df.csv', index=False)
pd.read_csv('test_df.csv')


,shr_score,tksh_distance,tksh_angle,shr_tkfit_dedx_max,trkfit,trkpid,subcluster,shrmoliereavg,trkshrhitdist2,hits_ratio,...,shr_tkfit_2cm_dedx_U,shr_tkfit_2cm_dedx_V,shr_tkfit_2cm_dedx_Y,shr_tkfit_gap10_dedx_U,shr_tkfit_gap10_dedx_V,shr_tkfit_gap10_dedx_Y,shrMCSMom,DeltaRMS2h,shrPCA1CMed_5cm,CylFrac2h_1cm
0,0.060575,18.913317,-4.230924e-01,3.801373,0.333333,-0.301138,11,12.420587,12.909738,0.686099,...,4.518741,-3.402824e+38,5.315952e+00,2.160177,-3.402824e+38,4.244851e+00,66.594580,2.284401,0.916764,0.265823
1,0.164992,979.931640,-3.402824e+38,4.410536,0.278689,NaN,9,4.436848,-1.000000,1.000000,...,4.410536,3.194422e+00,5.783832e+00,3.542191,3.786885e+00,8.806034e+00,84.530720,1.634778,0.945253,0.380952
2,0.059947,21.079191,-4.609286e-01,4.436085,0.241546,0.551929,3,6.491850,17.001177,0.769517,...,3.892681,3.675340e+00,1.527958e+00,3.244662,5.104007e+00,2.351186e+00,64.836075,0.754496,0.928969,0.842105
3,0.464402,23.760128,-6.125049e-01,4.500454,0.554913,-0.687969,8,5.091989,5.137548,0.582868,...,3.329740,3.543460e+00,-3.402824e+38,4.762031,3.927620e+00,-3.402824e+38,83.340910,0.651804,0.996718,0.880000
4,0.113901,55.102726,6.791702e-01,2.444770,0.640719,0.928477,3,6.599441,48.592247,0.164370,...,2.302901,2.772507e+00,2.326897e+00,2.783712,3.844409e+00,2.793659e+00,32.812866,2.275570,0.996513,0.103448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.008176,6.881827,3.203493e-01,4.140000,0.219057,0.948853,28,6.634148,6.404014,0.569033,...,4.586287,5.191947e+00,3.978160e+00,3.534937,4.625525e+00,4.039353e+00,120.146130,6.285063,0.808459,0.000000
996,0.002178,136.825680,-3.402824e+38,2.176004,0.105548,NaN,22,7.937480,-1.000000,1.000000,...,0.152080,2.002775e+00,2.186585e+00,-0.008714,2.629586e+00,2.188087e+00,208.518310,2.094876,0.727142,0.101770
997,0.030994,7.351628,3.889772e-01,2.261708,0.207237,0.648987,9,10.511190,2.278313,0.568224,...,4.155707,1.508573e+00,2.357685e+00,2.033264,4.154612e+00,3.306931e+00,33.613930,2.566018,0.688182,0.147287
998,0.043064,3.401190,5.817503e-01,5.047697,0.212851,-0.203747,10,40.429802,1.591751,0.454195,...,7.046705,6.228459e+00,-3.402824e+38,-0.008615,5.526859e+00,-3.402824e+38,63.576584,10.908969,0.849003,0.000000


### Define function to add scores to a dataframe (to be used in either version)

In [5]:
import os
import pickle

import xgboost as xgb

def load_booster(file_path):
    # Depending on the file extension, we want to either unpickle or load using load_model
    file_extension = file_path.split(".")[-1]
    if file_extension == "pickle":
        with open(file_path, "rb") as booster_file:
            booster = pickle.load(booster_file)
    elif file_extension in ["bin", "json"]:
        booster = xgb.Booster()
        booster.load_model(file_path)
    else:
        raise ValueError("File extension not supported")
    return booster

def add_bdt_scores(df, base_path, file_extension):
    TRAINVAR = [
        "shr_score",
        "tksh_distance",
        "tksh_angle",
        "shr_tkfit_dedx_max",
        "trkfit",
        "trkpid",
        "subcluster",
        "shrmoliereavg",
        "trkshrhitdist2",
        "hits_ratio",
        "secondshower_Y_nhit",
        "secondshower_Y_vtxdist",
        "secondshower_Y_dot",
        "anglediff_Y",
        "CosmicIPAll3D",
        "CosmicDirAll3D",
    ]

    LABELS = ["pi0", "nonpi0"]
    for label in LABELS:
        booster_file = os.path.join(base_path, f"booster_{label}_0304_extnumi.{file_extension}")
        booster = load_booster(booster_file)
        df[label + "_score"] = booster.predict(xgb.DMatrix(df[TRAINVAR]), ntree_limit=booster.best_iteration)

    # 0p BDT

    TRAINVARZP = [
        "shrmoliereavg",
        "shr_score",
        "trkfit",
        "subcluster",
        "CosmicIPAll3D",
        "CosmicDirAll3D",
        "secondshower_Y_nhit",
        "secondshower_Y_vtxdist",
        "secondshower_Y_dot",
        "anglediff_Y",
        "secondshower_V_nhit",
        "secondshower_V_vtxdist",
        "secondshower_V_dot",
        "anglediff_V",
        "secondshower_U_nhit",
        "secondshower_U_vtxdist",
        "secondshower_U_dot",
        "anglediff_U",
        "shr_tkfit_2cm_dedx_U",
        "shr_tkfit_2cm_dedx_V",
        "shr_tkfit_2cm_dedx_Y",
        "shr_tkfit_gap10_dedx_U",
        "shr_tkfit_gap10_dedx_V",
        "shr_tkfit_gap10_dedx_Y",
        "shrMCSMom",
        "DeltaRMS2h",
        "shrPCA1CMed_5cm",
        "CylFrac2h_1cm",
    ]

    LABELSZP = ["bkg"]
    for label in LABELSZP:
        booster_file = os.path.join(base_path, f"booster_{label}_0304_extnumi_vx.{file_extension}")
        booster = load_booster(booster_file)
        df[label + "_score"] = booster.predict(xgb.DMatrix(df[TRAINVARZP]), ntree_limit=booster.best_iteration)


### Run this part with xgboost 0.90

In [19]:
test_df = pd.read_csv('test_df.csv')
add_bdt_scores(test_df, "../../pickles/", "pickle")

In [21]:
test_df[['pi0_score', 'nonpi0_score', 'bkg_score']]

,pi0_score,nonpi0_score,bkg_score
0,0.089652,0.099982,0.434700
1,0.236882,0.265561,0.203931
2,0.032322,0.043380,0.087396
3,0.021060,0.019035,0.037246
4,0.000899,0.000820,0.081080
...,...,...,...
995,0.002789,0.003500,0.025714
996,0.057198,0.081968,0.168399
997,0.063193,0.079599,0.140482
998,0.014241,0.016344,0.059734


In [22]:
test_df.to_csv('test_df_xgb_version_0.90.csv', index=False)

In [18]:
import glob

pickle_files = glob.glob("../../pickles/*.pickle")
print(pickle_files)


['../../pickles/booster_pi0_0304_extnumi.pickle', '../../pickles/booster_bkg_0304_extnumi_vx.pickle', '../../pickles/booster_nonpi0_0304_extnumi.pickle', '../../pickles/booster_nonpi0_0304.pickle', '../../pickles/booster_pi0_0304.pickle']


In [23]:
import os
import pickle
import xgboost as xgb

for file_path in pickle_files:
    with open(file_path, "rb") as booster_file:
        booster = pickle.load(booster_file)
    new_file_path = os.path.splitext(file_path)[0] + ".bin"
    booster.save_model(new_file_path)


### Run the following part with the new xgboost version (here: 1.7.0)

Execute the cell above where we define the function that loads the models and applies the score

In [4]:
import pandas as pd

test_df = pd.read_csv('test_df.csv')
test_df

,shr_score,tksh_distance,tksh_angle,shr_tkfit_dedx_max,trkfit,trkpid,subcluster,shrmoliereavg,trkshrhitdist2,hits_ratio,...,shr_tkfit_2cm_dedx_U,shr_tkfit_2cm_dedx_V,shr_tkfit_2cm_dedx_Y,shr_tkfit_gap10_dedx_U,shr_tkfit_gap10_dedx_V,shr_tkfit_gap10_dedx_Y,shrMCSMom,DeltaRMS2h,shrPCA1CMed_5cm,CylFrac2h_1cm
0,0.060575,18.913317,-4.230924e-01,3.801373,0.333333,-0.301138,11,12.420587,12.909738,0.686099,...,4.518741,-3.402823e+38,5.315952e+00,2.160177,-3.402823e+38,4.244851e+00,66.594580,2.284401,0.916764,0.265823
1,0.164992,979.931640,-3.402823e+38,4.410536,0.278689,NaN,9,4.436848,-1.000000,1.000000,...,4.410536,3.194422e+00,5.783832e+00,3.542191,3.786885e+00,8.806034e+00,84.530720,1.634778,0.945253,0.380952
2,0.059947,21.079191,-4.609286e-01,4.436085,0.241546,0.551929,3,6.491850,17.001177,0.769517,...,3.892681,3.675340e+00,1.527958e+00,3.244662,5.104007e+00,2.351186e+00,64.836075,0.754496,0.928969,0.842105
3,0.464402,23.760128,-6.125049e-01,4.500453,0.554913,-0.687969,8,5.091989,5.137548,0.582868,...,3.329741,3.543460e+00,-3.402823e+38,4.762031,3.927620e+00,-3.402823e+38,83.340910,0.651804,0.996718,0.880000
4,0.113901,55.102726,6.791702e-01,2.444770,0.640719,0.928477,3,6.599442,48.592247,0.164370,...,2.302901,2.772507e+00,2.326897e+00,2.783712,3.844409e+00,2.793659e+00,32.812866,2.275570,0.996513,0.103448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.008176,6.881827,3.203493e-01,4.140000,0.219057,0.948853,28,6.634148,6.404014,0.569033,...,4.586287,5.191948e+00,3.978160e+00,3.534937,4.625525e+00,4.039353e+00,120.146130,6.285063,0.808459,0.000000
996,0.002178,136.825680,-3.402823e+38,2.176004,0.105548,NaN,22,7.937480,-1.000000,1.000000,...,0.152080,2.002775e+00,2.186585e+00,-0.008714,2.629586e+00,2.188087e+00,208.518310,2.094876,0.727142,0.101770
997,0.030994,7.351628,3.889772e-01,2.261708,0.207237,0.648987,9,10.511190,2.278313,0.568224,...,4.155707,1.508573e+00,2.357685e+00,2.033264,4.154612e+00,3.306931e+00,33.613930,2.566018,0.688182,0.147287
998,0.043064,3.401190,5.817503e-01,5.047697,0.212851,-0.203747,10,40.429802,1.591751,0.454195,...,7.046705,6.228459e+00,-3.402823e+38,-0.008615,5.526859e+00,-3.402823e+38,63.576584,10.908969,0.849003,0.000000


In [6]:
add_bdt_scores(test_df, "../../pickles/", "bin")
test_df[['pi0_score', 'nonpi0_score', 'bkg_score']]

[12:06:53] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[12:06:53] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[12:06:53] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
 

,pi0_score,nonpi0_score,bkg_score
0,0.089652,0.099982,0.434700
1,0.236882,0.265561,0.203931
2,0.032322,0.043380,0.087396
3,0.021060,0.019035,0.037246
4,0.000899,0.000820,0.081080
...,...,...,...
995,0.002789,0.003500,0.025714
996,0.057198,0.081968,0.168399
997,0.063193,0.079599,0.140482
998,0.014241,0.016344,0.059734


In [7]:
test_df_old_scores = pd.read_csv('test_df_xgb_version_0.90.csv')
test_df_old_scores[['pi0_score', 'nonpi0_score', 'bkg_score']]

,pi0_score,nonpi0_score,bkg_score
0,0.089652,0.099982,0.434700
1,0.236882,0.265561,0.203931
2,0.032322,0.043380,0.087396
3,0.021060,0.019035,0.037246
4,0.000899,0.000820,0.081080
...,...,...,...
995,0.002789,0.003500,0.025714
996,0.057198,0.081968,0.168399
997,0.063193,0.079599,0.140482
998,0.014241,0.016344,0.059734


In [10]:
# Test if scores with the new version are the same as with the old version
import numpy as np

np.testing.assert_array_almost_equal(test_df[['pi0_score', 'nonpi0_score', 'bkg_score']].values, test_df_old_scores[['pi0_score', 'nonpi0_score', 'bkg_score']].values, decimal=5)

In [13]:
import os
import glob
import xgboost as xgb

for file_path in glob.glob("../../pickles/*.bin"):
    booster = xgb.Booster()
    booster.load_model(file_path)
    new_file_path = os.path.splitext(file_path)[0] + ".json"
    booster.save_model(new_file_path)


[12:09:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[12:09:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[12:09:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[12:09:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility
[12:09:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667414656755/work/src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


In [14]:
add_bdt_scores(test_df, "../../pickles/", "json")
test_df[['pi0_score', 'nonpi0_score', 'bkg_score']]

/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/trettin/miniconda/envs/xgb/lib/python3.11/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
 

,pi0_score,nonpi0_score,bkg_score
0,0.089652,0.099982,0.434700
1,0.236882,0.265561,0.203931
2,0.032322,0.043380,0.087396
3,0.021060,0.019035,0.037246
4,0.000899,0.000820,0.081080
...,...,...,...
995,0.002789,0.003500,0.025714
996,0.057198,0.081968,0.168399
997,0.063193,0.079599,0.140482
998,0.014241,0.016344,0.059734


In [15]:
# Test if scores with the JSON version are the same as with the old version
import numpy as np

np.testing.assert_array_almost_equal(test_df[['pi0_score', 'nonpi0_score', 'bkg_score']].values, test_df_old_scores[['pi0_score', 'nonpi0_score', 'bkg_score']].values, decimal=5)